In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
match=pd.read_csv('matches.csv')
match.columns

Index(['id', 'Season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2',
       'umpire3'],
      dtype='object')

In [3]:
delivery=pd.read_csv('deliveries.csv')
delivery.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')

In [4]:
total_score_df=delivery.groupby(['match_id','inning']).sum()['total_runs'].reset_index()

In [5]:
total_score_df=total_score_df[total_score_df['inning']==1]

In [6]:
match_df=match.merge(total_score_df[['match_id','total_runs']],left_on='id',right_on='match_id')

In [7]:
match_df['team2'].unique()

array(['Royal Challengers Bangalore', 'Rising Pune Supergiant',
       'Kolkata Knight Riders', 'Kings XI Punjab', 'Delhi Daredevils',
       'Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions',
       'Rajasthan Royals', 'Chennai Super Kings', 'Deccan Chargers',
       'Pune Warriors', 'Kochi Tuskers Kerala', 'Rising Pune Supergiants',
       'Delhi Capitals'], dtype=object)

In [8]:
match_df['team1']=match_df['team1'].replace({'Delhi Daredevils':'Delhi Capitals',
                                              'Deccan Chargers':'Sunrisers Hyderabad'})

In [9]:
match_df['team2']=match_df['team2'].replace({'Delhi Daredevils':'Delhi Capitals',
                                              'Deccan Chargers':'Sunrisers Hyderabad'})

In [10]:
teams=['Chennai Super Kings','Royal Challengers Bangalore','Kolkata Knight Riders','Kings XI Punjab','Delhi Capitals','Sunrisers Hyderabad','Mumbai Indians',
       'Rajasthan Royals']

match_df=match_df[match_df['team1'].isin(teams)]
match_df=match_df[match_df['team2'].isin(teams)]

In [11]:
match_df=match_df[match_df['dl_applied']==0]

In [12]:
match_df=match_df[match_df['result']=='normal']

In [13]:
match_df=match_df[['match_id','city','winner','total_runs']]

In [14]:
Deliver_df=match_df.merge(delivery,on='match_id')
Deliver_df.columns

Index(['match_id', 'city', 'winner', 'total_runs_x', 'inning', 'batting_team',
       'bowling_team', 'over', 'ball', 'batsman', 'non_striker', 'bowler',
       'is_super_over', 'wide_runs', 'bye_runs', 'legbye_runs', 'noball_runs',
       'penalty_runs', 'batsman_runs', 'extra_runs', 'total_runs_y',
       'player_dismissed', 'dismissal_kind', 'fielder'],
      dtype='object')

In [15]:
Deliver_df=Deliver_df[Deliver_df['inning']==2]

In [16]:
Deliver_df['current_score']=Deliver_df.groupby(['match_id'])['total_runs_y'].cumsum()

In [17]:
Deliver_df['runs_left']=(Deliver_df['total_runs_x']+1)-Deliver_df['current_score']

In [18]:
Deliver_df['balls_left']=120-((Deliver_df['over']-1)*6+(Deliver_df['ball']))

In [19]:
Deliver_df['RRR']=((Deliver_df['runs_left']/Deliver_df['balls_left'].replace(0, 1))*6).round(2)

In [26]:
Deliver_df['ball_played']=120-Deliver_df['balls_left']

In [27]:
Deliver_df['CRR']=((Deliver_df['current_score']/Deliver_df['ball_played'])*6).round(2)

In [20]:
Deliver_df['player_dismissed']=Deliver_df['player_dismissed'].fillna(0)

In [21]:
Deliver_df['player_dismissed']=Deliver_df['player_dismissed'].apply(lambda x:x if x==0 else 1)

In [22]:
wicket=Deliver_df.groupby('match_id')['player_dismissed'].cumsum()
Deliver_df['wicket_left']=10-wicket

In [28]:
Deliver_df['result']=Deliver_df['batting_team']==Deliver_df['winner']
Deliver_df['result'] = Deliver_df['result'].astype(int)
Deliver_df

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,dismissal_kind,fielder,current_score,runs_left,balls_left,RRR,wicket_left,result,ball_played,CRR
125,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,NaN,NaN,1,207,119,10.44,10,0,1,6.00
126,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,NaN,NaN,1,207,118,10.53,10,0,2,3.00
127,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,...,NaN,NaN,1,207,117,10.62,10,0,3,2.00
128,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,...,NaN,NaN,3,205,116,10.60,10,0,4,4.50
129,1,Hyderabad,Sunrisers Hyderabad,207,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,5,Mandeep Singh,...,NaN,NaN,7,201,115,10.49,10,0,5,8.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147193,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,2,RA Jadeja,...,NaN,NaN,152,1,4,1.50,5,0,116,7.86
147194,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,3,SR Watson,...,NaN,NaN,154,-1,3,-2.00,5,0,117,7.90
147195,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,4,SR Watson,...,run out,KH Pandya,155,-2,2,-6.00,4,0,118,7.88
147196,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,5,SN Thakur,...,NaN,NaN,157,-4,1,-24.00,4,0,119,7.92


In [29]:
final_df = Deliver_df[[
    'batting_team',
    'bowling_team',
    'city',
    
    'balls_left',
    'runs_left',
    'wicket_left',
    'total_runs_x',
    'CRR',
    'RRR',
    'result'
]]

In [30]:
final_df=final_df.sample(final_df.shape[0]).reset_index(drop=True)

In [31]:
final_df

,batting_team,bowling_team,city,balls_left,runs_left,wicket_left,total_runs_x,CRR,RRR,result
0,Rajasthan Royals,Delhi Daredevils,Jaipur,117,145,10,151,14.00,7.44,1
1,Rajasthan Royals,Mumbai Indians,Jaipur,41,58,7,171,8.66,8.49,1
2,Royal Challengers Bangalore,Sunrisers Hyderabad,Bangalore,19,9,7,161,9.09,2.84,1
3,Royal Challengers Bangalore,Deccan Chargers,Mumbai,41,5,9,82,5.92,0.73,1
4,Chennai Super Kings,Delhi Daredevils,Delhi,101,159,10,187,9.16,9.45,1
...,...,...,...,...,...,...,...,...,...,...
71383,Mumbai Indians,Chennai Super Kings,Chennai,112,159,10,165,5.25,8.52,0
71384,Royal Challengers Bangalore,Rajasthan Royals,Ahmedabad,45,34,9,130,7.76,4.53,1
71385,Rajasthan Royals,Mumbai Indians,Mumbai,44,88,7,197,8.68,12.00,0
71386,Kolkata Knight Riders,Mumbai Indians,Mumbai,83,135,8,187,8.59,9.76,0


In [34]:
final_df.isnull().sum()

batting_team    0
bowling_team    0
city            0
balls_left      0
runs_left       0
wicket_left     0
total_runs_x    0
CRR             0
RRR             0
result          0
dtype: int64

In [35]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression  # or RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle


X = final_df.drop(['result'], axis=1)
y = final_df['result']


categorical_cols = ['batting_team', 'bowling_team', 'city']


preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_cols)
    ],
    remainder='passthrough'  
)

# 4️⃣ Create pipeline: preprocessing + model
pipe = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', LogisticRegression(max_iter=1000))  
])

# 5️⃣ Train/test split and model training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipe.fit(X_train, y_train)


with open('pip.pkl', 'wb') as f:
    pickle.dump(pipe, f)

print("✅ Pipeline trained and saved as pip.pkl")


✅ Pipeline trained and saved as pip.pkl
